# A Machine Learning Framework for Stroke Prediction: Balancing Precision and Recall in Healthcare Analytics (Master Notebook)

In [ ]:
import papermill
import scrapbook
from itertools import product
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import joblib
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import math



In [ ]:
dataset = pd.read_csv('healthcare-dataset-stroke-data.csv')
dataset

## Data Visualization

In [ ]:
sns.set(style='whitegrid', context='talk')

stroke_data = dataset[dataset['stroke'] == 1].drop('id', axis=1).describe().T
no_stroke_data = dataset[dataset['stroke'] == 0].drop('id', axis=1).describe().T

cmap = sns.diverging_palette(220, 20, as_cmap=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

sns.heatmap(stroke_data[['mean']], annot=True, cmap=cmap, linewidths=0.8,
            linecolor='gray', cbar=True, fmt='.2f', ax=axes[0],
            annot_kws={"size":14})
axes[0].set_title('Stroke Suffered', fontsize=16, pad=16)
axes[0].set_ylabel('')
axes[0].set_xlabel('')
axes[0].tick_params(labelsize=12)

sns.heatmap(no_stroke_data[['mean']], annot=True, cmap=cmap, linewidths=0.8,
            linecolor='gray', cbar=True, fmt='.2f', ax=axes[1],
            annot_kws={"size":14})
axes[1].set_title('No Stroke Suffered', fontsize=16, pad=16)
axes[1].set_ylabel('')
axes[1].set_xlabel('')
axes[1].tick_params(labelsize=12)

fig.suptitle('Mean Statistics Comparison for Stroke Data', fontsize=18, y=1.05)
plt.tight_layout()
plt.show()

### From this insight, we can notice that people that are more likely to suffer from a stroke are the ones that are elder and have a high averge glucose level, indicating diabetes. Moreover, heart disease and hypertension can contribute to being affected by a stroke in the near future. Thus, BMI doesn't seem as relevant as the other features.

In [ ]:
counts = dataset['stroke'].value_counts()
labels = ['No Stroke Suffered', 'Stroke Suffered']
percentages = [counts[0] / counts.sum() * 100, counts[1] / counts.sum() * 100]

colors = ['#66b3ff', '#ff9999']
explode = (0.1, 0)

fig, ax = plt.subplots(figsize=(10, 7))

wedges, texts, autotexts = ax.pie(
    percentages,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    explode=explode,
    colors=colors,
    shadow=True,
    wedgeprops={'edgecolor': 'black', 'linewidth': 1}
)

ax.set_title('Stroke Events (%)', fontsize=16)
plt.tight_layout()
plt.show()

### We are dealing with a highly imbalanced dataset, where only 4.9% of the samples are from people that have suffered a stroke.

## Data Cleaning

In [ ]:
print(dataset.isnull().sum())

### We have 201 nulls in the BMI column that we have to deal with. We have several options: remove the affected rows, remove the affected columns, replace with the mean, or train a model to predict the missing BMI. Let's take a look at the mean of stroke and non-stroke data.

In [ ]:
dataset['bmi'] = dataset['bmi'].fillna(dataset['bmi'].mean())

In [ ]:
print(dataset.duplicated().sum())

### There are no duplicated rows in the dataset.

In [ ]:
dataset

### The ID column should also be removed as it doesn't pose relevancy to the training process.

In [ ]:
dataset.drop('id', axis=1, inplace=True)

In [ ]:
dataset

In [ ]:
categorical_features = [
    col for col in dataset.columns 
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10 
]


numerical_features = [col for col in dataset.columns if col not in categorical_features]

categorical_features = [dataset.columns.get_loc(col) for col in categorical_features]
numerical_features = [dataset.columns.get_loc(col) for col in numerical_features]
categorical_features, numerical_features

In [ ]:
categorical_cols = [
    col for col in dataset.columns
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10
]

plots_per_row = 3
rows = math.ceil(len(categorical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))

axes = axes if isinstance(axes, np.ndarray) else [[axes]]

for i, col in enumerate(categorical_cols):
    r = i // plots_per_row  
    c = i % plots_per_row   
    sns.countplot(x=col, data=dataset, palette='viridis', ax=axes[r][c])
    axes[r][c].set_title(f"Distribution of {col}")
    axes[r][c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i+1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r][c].set_visible(False)

fig.tight_layout()
plt.show()

### From here, we can think of some feature engineering steps. For instance, there is one individual with 'Other' gender category. Since there isn't a lot of data for the 'Other' category, it will pose extra challange for a model. Thus, the 'Other' individual has been removed.

In [ ]:
dataset = dataset[dataset['gender'] != 'Other']

In [ ]:
numerical_cols = [
    col for col in dataset.columns
    if np.issubdtype(dataset[col].dtype, np.number) and dataset[col].nunique() > 10
]


plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes)  

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    sns.histplot(data=dataset, x=col, kde=True, ax=axes[r, c], color='skyblue')
    axes[r, c].set_title(f"Distribution of {col}")
    axes[r, c].set_xlabel(col)

total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
numerical_cols = [
    col for col in dataset.columns
    if np.issubdtype(dataset[col].dtype, np.number) and dataset[col].nunique() > 10
]

plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes) 

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    sns.histplot(data=dataset, x=col, kde=True, ax=axes[r, c], color='skyblue')
    axes[r, c].set_title(f"Distribution of {col}")
    axes[r, c].set_xlabel(col)

total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

### Specifically, we can check the distribution relative to the two classes (stroke vs non-stroke).

In [ ]:
categorical_cols = [
    col for col in dataset.columns
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10
]

plots_per_row = 3
rows = math.ceil(len(categorical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = axes if isinstance(axes, np.ndarray) else [[axes]]

for i, col in enumerate(categorical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    
    sns.countplot(x=col, hue='stroke', data=dataset, palette='viridis', ax=axes[r][c])
    axes[r][c].set_title(f"Distribution of {col} by Stroke Status")
    axes[r][c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i+1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r][c].set_visible(False)

fig.tight_layout()
plt.show()

### Having the 'children' and 'never_worked' category at the same time can be tricky. That's because never worked doesn't seem to have a lot of data in general, while there are more records of children. Moreover, childrens can also be considered to be people that never worked. So, a decision can be merging the children and never_worked type for the work_type class into one: never_worked. In essence, this difference between individuals can also be made via the 'age' category.

In [ ]:
dataset[(dataset['work_type'] == 'children') & (dataset['ever_married'] == 'Yes')]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='age', hue='work_type', multiple='stack', palette='viridis')
plt.title('Distribution of Work Type by Age')
plt.show()


### We can also notice this in the graph above. Therefore, a merging between 'Children' and 'Never Worked' is valid.

In [ ]:
dataset.loc[(dataset['work_type'] == 'children'), 'work_type'] = 'Never_worked'

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='age', hue='work_type', multiple='stack', palette='viridis')
plt.title('Distribution of Work Type by Age')
plt.show()

In [ ]:
numerical_cols = [
    col for col in dataset.columns 
    if dataset[col].dtype != 'object' and dataset[col].nunique() > 10 and 'id' not in col.lower()
]

plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes) 

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    
    sns.histplot(data=dataset, x=col, hue='stroke', palette='viridis', ax=axes[r, c], kde=True)
    axes[r, c].set_title(f"Distribution of {col} by Stroke Status")
    axes[r, c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

fig.tight_layout()
plt.show()

## Splitting the data for the workers

### Creating the train-val-test sets

In [ ]:
dataset

In [ ]:
def train_val_test_split(X, y, test_size=0.2, val_size=0.2, random_state=42):

    temp_size = test_size + val_size
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=temp_size, stratify=y, random_state=random_state
    )
    
    test_frac = test_size / temp_size
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=test_frac, stratify=y_temp, random_state=random_state
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X = dataset.drop(columns=['stroke'])
y = dataset['stroke']

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, test_size=0.15, val_size=0.15)

In [ ]:
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_val = pd.concat([X_val, y_val], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

dataset_train.to_csv('dataset_train.csv', index=False)
dataset_val.to_csv('dataset_val.csv', index=False)
dataset_test.to_csv('dataset_test.csv', index=False)

## Papermill setup

In [ ]:
def execute_notebook(*args, parameters={}, **kwargs):
    return papermill.execute_notebook(*args, parameters=parameters, **kwargs)

In [ ]:
name = 'final_models'

models_path = 'workers/models/' + name
workers_path = 'workers/notebooks/' + name

for path in [models_path, workers_path]:
    if not os.path.exists(path):
        os.makedirs(path)

imbalanced_action = [0, 1, 2, 3, 4]
use_PCA = [False]
scale = [False, True]
normalize = [False, True]
feature_binning = [False, True]
feature_selection_mode = [0, 1, 2]
k_features = [2, 3, 4]
threshold = [0.5]
lr = [0.01, 0.005, 0.001, 0.0005, 0.0001]
model_type = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29]
loss_type = [0, 2, 3, 4, 5, 6]

parameters = set() 

for imb in imbalanced_action:
    for pca in use_PCA:
        for sc in scale:
            for nm in normalize:
                for fe in feature_binning:
                    for fs in feature_selection_mode:
                        for k in k_features:
                            for t in threshold:
                                for mt in model_type:
                                    for learning_rate in lr:
                                        for lt in loss_type:
                                            if fe == True:
                                                nm = False
                                                sc = False
                                            if fs == 2:
                                                k = 0
                                            bp = {
                                                'imbalanced_action': imb,
                                                'use_PCA': pca,
                                                'scale': sc,
                                                'normalize': nm,
                                                'feature_binning': fe,
                                                'feature_selection_mode': fs,
                                                'k_features': k,
                                                'threshold': t,
                                                'model_type': mt,
                                                'learning_rate': learning_rate,
                                                'loss_type': lt
                                            }
                                            parameters.add(frozenset(bp.items()))

In [ ]:
len(parameters)

## Models training

In [ ]:
with ThreadPoolExecutor() as pool, tqdm(total=len(parameters)) as progress:
    for idx, param in enumerate(parameters):
        if idx > 55928:
            param = dict(param)
            param['model_file_path'] = f'{models_path}/stroke_model_{idx}.pkl'
            param['pca_file_path'] = f'{models_path}/stroke_pca_{idx}.pkl'
            param['scaler_file_path'] = f'{models_path}/stroke_scaler_{idx}.pkl'
            param['normalizer_file_path'] = f'{models_path}/stroke_normalizer_{idx}.pkl'
            param['encoder_file_path'] = f'{models_path}/stroke_encoder_{idx}.pkl' 
            execute_notebook(
                'stroke_detection_worker_best.ipynb',
                f'{workers_path}/stroke_model_{idx}.ipynb',
                parameters=param
            )

## Extracting results from workers

In [ ]:
def get_scrap(nb, scrap_name):
    return nb.scraps[scrap_name].data if scrap_name in nb.scraps else None

In [ ]:
fieldnames = [
    "parameters",
    "precision_train",
    "recall_train",
    "f1_train",
    "accuracy_train",
    "precision_val",
    "recall_val",
    "f1_val",
    "accuracy_val",
    "pr_auc_val",
    "precision_optimal",
    "recall_optimal",
    "f1_optimal",
    "mcc_optimal",
    "precision_optimal_f2",
    "recall_optimal_f2",
    "f2_optimal",
    "mcc_optimal_f2",
    "precision_test",
    "recall_test",
    "f1_test",
    "accuracy_test",
    "f2_test",
    "mcc_test",
]

In [ ]:
import csv
import json

directory_path = workers_path
output_csv_path = "results.csv"

with open(output_csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for filename in os.listdir(directory_path):
        if not filename.endswith(".ipynb"):
            continue

        notebook_path = os.path.join(directory_path, filename)
        try:
            nb = scrapbook.read_notebook(notebook_path)
            
            row = {}
            row["parameters"] = json.dumps(nb.parameters)
            
            row["precision_train"] = get_scrap(nb, "precision_train")
            row["recall_train"]    = get_scrap(nb, "recall_train")
            row["f1_train"]        = get_scrap(nb, "f1_score_train")
            row["accuracy_train"]  = get_scrap(nb, "accuracy_score_train")

            row["precision_val"]   = get_scrap(nb, "precision")
            row["recall_val"]      = get_scrap(nb, "recall")
            row["f1_val"]          = get_scrap(nb, "f1_score")
            row["accuracy_val"]    = get_scrap(nb, "accuracy")
            row["pr_auc_val"]      = get_scrap(nb, "pr_auc")

            row["precision_optimal"] = get_scrap(nb, "precision_optimal_f1")
            row["recall_optimal"]    = get_scrap(nb, "recall_optimal_f1")
            row["f1_optimal"]        = get_scrap(nb, "f1_score_optimal")
            row["mcc_optimal"]       = get_scrap(nb, "mcc_optimal_f1")

            row["precision_optimal_f2"] = get_scrap(nb, "precision_optimal_f2")
            row["recall_optimal_f2"]    = get_scrap(nb, "recall_optimal_f2")
            row["f2_optimal"]           = get_scrap(nb, "f2_score_optimal_thres")
            row["mcc_optimal_f2"]       = get_scrap(nb, "mcc_optimal_f2")

            row["precision_test"] = get_scrap(nb, "precision_test")
            row["recall_test"]    = get_scrap(nb, "recall_test")
            row["f1_test"]        = get_scrap(nb, "f1_score_test")
            row["accuracy_test"]  = get_scrap(nb, "accuracy_score_test")
            row["f2_test"]        = get_scrap(nb, "f2_score_test")
            row["mcc_test"]       = get_scrap(nb, "mcc_test")

            writer.writerow(row)

        except Exception as e:
            print(f"Error reading {filename}: {e}")

In [ ]:
all_data = pd.read_csv('results.csv')

In [ ]:
all_data

In [ ]:
results = all_data

In [ ]:
import json
import pandas as pd

results["params_dict"] = results["parameters"].apply(json.loads)        
results = results.join(pd.json_normalize(results["params_dict"]))          
results.drop(columns=["parameters", "params_dict"], inplace=True)   

In [ ]:
results_without_model_path = results

In [ ]:
results_without_model_path = results_without_model_path.sort_values(by='f2_optimal', ascending=False)

In [ ]:
results_without_model_path

In [ ]:
print(results_without_model_path.to_string())

In [ ]:
results_best = results_without_model_path[
    (results_without_model_path['f2_test'] >= results_without_model_path['f2_optimal']) &
     (results_without_model_path['recall_optimal_f2'] >= 0.7)
]

results_best

In [ ]:
print(results_best.to_string())